<a href="https://colab.research.google.com/github/tuhinagupta/15640_DS/blob/master/final_attack_defense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
from PIL import Image
from torch.utils import data

import pandas as pd
from keras import regularizers
import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score

LOADING SAVED MODELS

In [ ]:
base_classifier = "saved_models/base_classifier_resnet.pth"
attack_model = "saved_models/attack_model_vgg.pth"
fgsm_trained_model = "saved_models/fgsm_trained_resnet.pth"
pgd_trained_model = "saved_models/pgd_trained_resnet.pth"
fgsm_pgd_trained_model = "saved_models/fgsm_pgd_trained_resnet.pth"

In [ ]:
''' Data loading and augmentation '''
transform = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])   # [torchvision.transforms.Resize(size = (224,224)),

transform1 = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])   # [torchvision.transforms.Resize(size = (224,224)),
     
transform2 = torchvision.transforms.Compose([
     torchvision.transforms.RandomHorizontalFlip(1),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])   # [torchvision.transforms.Resize(size = (224,224)),

trainset1 = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform1)
trainset2 = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform2)

trainset = torch.utils.data.ConcatDataset([trainset1, trainset2])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


CLASSIFIER /DEFENSE MODEL

In [ ]:
'''Basic building block 1 for ResNet18 model'''
class BasicBlock(nn.Module):
    def __init__(self,channel_size1, channel_size2, stride1=1, stride2=2):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(channel_size1, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(channel_size2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2 = nn.Conv2d(channel_size2, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_size2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.shortcut = nn.Conv2d(channel_size1, channel_size2, kernel_size=1, stride=stride1, bias=False)


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
'''Basic building block 2 for ResNet18 model'''
class BasicBlock2(nn.Module):

    def __init__(self, channel_size, stride=1):
        super(BasicBlock2, self).__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_size, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_size, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.shortcut = nn.Conv2d(channel_size, channel_size, kernel_size=1, stride=stride, bias=False)
        self.conv2 = nn.Conv2d(channel_size, channel_size, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_size, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
''' ResNet18 model architecture for a 32x32 input image '''
class Network(nn.Module):
    def __init__(self, num_feats, hidden_sizes, num_classes, feat_dim=10):
        super(Network, self).__init__()
        

        self.hidden_sizes = [num_feats] + hidden_sizes + [num_classes]
        
        self.layers = []
        for idx, channel_size in enumerate(hidden_sizes):
            if idx==0:
              self.layers.append(nn.Conv2d(in_channels=self.hidden_sizes[idx], 
                                            out_channels=self.hidden_sizes[idx+1], 
                                            kernel_size=3, stride=1, padding = 1, bias=False))
              self.layers.append(nn.BatchNorm2d(num_features=self.hidden_sizes[idx+1],eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
              self.layers.append(nn.ReLU(inplace=True))
              
              self.layers.append(nn.MaxPool2d(3, stride=2, padding=1, dilation=1, ceil_mode=False))
              # 2 times
              self.layers.append(nn.Sequential(BasicBlock2(channel_size = channel_size),
                                                BasicBlock2(channel_size = channel_size)))
            else:
              self.layers.append(BasicBlock(channel_size1=self.hidden_sizes[idx], channel_size2=self.hidden_sizes[idx+1]))
              self.layers.append(BasicBlock2(channel_size=channel_size))

        self.layers.append(nn.AdaptiveAvgPool2d(output_size=(1,1)))
        self.layers = nn.Sequential(*self.layers)
        
        self.linear1 = nn.Linear(self.hidden_sizes[-2], self.hidden_sizes[-1], bias=False)
    
    def forward(self, x, evalMode=False):
        output = x
        output = self.layers(output)
        
        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = output.reshape(output.shape[0], output.shape[1])
        
        label_output = self.linear1(output)

        return label_output

# Convolutional and linear layer initializations 
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

ATTACK MODEL

In [ ]:
''' VGG16 basic block for 2 blocks '''
class BasicBlock2B(nn.Module):
    def __init__(self,channel_size1, channel_size2, stride1=1):
        super(BasicBlock2B, self).__init__()
        self.conv1 = nn.Conv2d(channel_size1, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(channel_size2, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_size2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

    def forward(self, x):
        out = F.relu(self.conv1(x), inplace=True)
        out = self.conv2(out)
        out = F.relu(out)
        mp = nn.MaxPool2d(2, stride=2, padding=0, dilation=1, ceil_mode=False)
        out = self.bn1(mp(out))
        dm = nn.Dropout(p=0.25)
        output = dm(out)
        return output

In [ ]:
''' VGG16 basic block for 3 blocks '''
class BasicBlock3B(nn.Module):
    def __init__(self,channel_size1, channel_size2, stride1=1):
        super(BasicBlock3B, self).__init__()
        self.conv1 = nn.Conv2d(channel_size1, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(channel_size2, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_size2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv3 = nn.Conv2d(channel_size2, channel_size2, kernel_size=3, stride=stride1, padding=1, bias=False)

    def forward(self, x):
        out = F.relu(self.conv1(x), inplace=True)
        out = F.relu(self.conv2(out), inplace=True)
        out = F.relu(self.conv3(out), inplace=True)
        mp = nn.MaxPool2d(2, stride=2, padding=0, dilation=1, ceil_mode=False)
        out = self.bn1(mp(out))
        
        dm = nn.Dropout(p=0.25)
        output = dm(out)
        return output

In [ ]:
''' VGG16 model architecture for 32x32 input image '''
class Network_Attack(nn.Module):
    def __init__(self, num_feats, hidden_sizes, num_classes, feat_dim=10):
        super(Network_Attack, self).__init__()

        self.hidden_sizes = [num_feats] + hidden_sizes + [num_classes]
        
        self.layers = []
        for idx, channel_size in enumerate(hidden_sizes):
          if idx == 0 or idx == 1:
            self.layers.append(BasicBlock2B(channel_size1=self.hidden_sizes[idx], channel_size2=self.hidden_sizes[idx+1]))
          if idx == 2:
            self.layers.append(BasicBlock3B(channel_size1=self.hidden_sizes[idx], channel_size2=self.hidden_sizes[idx+1]))
          if idx == 3:
            self.layers.append(BasicBlock3B(channel_size1=self.hidden_sizes[idx], channel_size2=self.hidden_sizes[idx+1]))
            self.layers.append(BasicBlock3B(channel_size1=self.hidden_sizes[idx+1], channel_size2=self.hidden_sizes[idx+1]))

        self.layers.append(nn.AdaptiveAvgPool2d(output_size=(7,7)))
        self.layers = nn.Sequential(*self.layers)
        
        self.linear1 = nn.Linear(self.hidden_sizes[-2], self.hidden_sizes[-1], bias=False)
        
    
    def forward(self, x, evalMode=False):
        output = x
        output = self.layers(output)
        
        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = output.reshape(output.shape[0], output.shape[1])
        
        label_output = self.linear1(output)

        return label_output

#Weights initializations
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

CLASSIFIER TRAINING


In [ ]:
''' Model training '''
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        accuracy = 0.0
        total = 0.0

        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()
            accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
            total += len(labels)

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
                
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
            del outputs 

        
        print("Accuracy in train", (accuracy/total)*100.0)
        if task == 'Classification':
            train_loss, train_acc = test_classify(model, data_loader)
            val_loss, val_acc = test_classify(model, test_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc*100.0, val_loss, val_acc*100.0))
            
        scheduler.step()

''' Function to return accuracy of model ''' 
def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats).detach()
        
        loss = criterion(outputs, labels)
        
        accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])

        del feats
        del labels
        del outputs

    model.train()
    return np.mean(test_loss), accuracy/total

In [ ]:
numEpochs = 32
num_feats = 3

learningRate = 0.15
weightDecay = 1e-3

hidden_sizes = [64, 128, 256, 512]
num_classes = len(trainset1.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = Network(num_feats, hidden_sizes, num_classes)
network.apply(init_weights)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.85)   

network.train()
network.to(device)
train(network, trainloader, testloader)

ATTACK MODEL TRAINING

In [ ]:
numEpochs = 40
num_feats = 3

learningRate = 1e-3
weightDecay = 5e-5

hidden_sizes = [64, 128, 256, 512]
num_classes = len(trainset1.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = Network_Attack(num_feats, hidden_sizes, num_classes)
network.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(network.parameters(), lr = 1e-3, weight_decay=weightDecay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.85)   

network.train()
network.to(device)
train(network, trainloader, testloader)

FGSM ATTACK

In [ ]:
''' FGSM attack code '''
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [-1,1] range
    perturbed_image = torch.clamp(perturbed_image, -1, 1)
    # Return the perturbed image
    return perturbed_image

In [ ]:
''' Generate adversarial images using FGSM attack '''
def attack_images(model, test_loader, epsilon):
    model.eval()
    correct = 0

    p_images = []
    p_label = []

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        feats.requires_grad = True

        outputs = model(feats)
        init_pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        if init_pred != labels.item():
            p_images.append(feats)
            p_label.append(labels)
            continue

        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()

        # Call FGSM Attack
        perturbed_data = fgsm_attack(feats, epsilon, feats.grad.data)
        p_images.append(perturbed_data)
        p_label.append(labels)
           
        del feats
        del labels
        del outputs

    return p_images,p_label

PGD ATTACK

In [ ]:
''' PGD attack adversarial images generation '''
def pgd_attack(image,epsilon,data_grad,alpha,steps,random_start):

  #make sure alpha*steps = epsilon
  
  adv_images = image
  #random initialization 
  if random_start == True:
    adv_images = adv_images + torch.empty_like(adv_images).uniform_(-epsilon,epsilon)
    adv_images = torch.clamp(adv_images,0, 1)

  for i in range(steps):
    adv_images = adv_images + alpha*data_grad.sign()
    delta = torch.clamp(adv_images - image,-epsilon,epsilon)
    adv_images = torch.clamp(image + delta,0, 1)

  return adv_images

In [ ]:
''' Train the model using PGD-generated attack images '''
def attack_images_pgd(model, test_loader):
    model.eval()
    correct = 0
### PGD attack values ###
    epsilon = 0.2
    random_start = True
    alpha = 1/255
    steps = 50
    p_images = []
    p_label = []

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        feats.requires_grad = True

        outputs = model(feats)
        init_pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        if init_pred != labels.item():
            p_images.append(feats)
            p_label.append(labels)
            continue

        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()

        # Call PGD Attack
        perturbed_data = pgd_attack(feats, epsilon, feats.grad.data, alpha, steps, random_start)
        p_images.append(perturbed_data)
        p_label.append(labels)
           
        del feats
        del labels
        del outputs

    return p_images,p_label

ADVERSARIAL TRAINING (FGSM)

In [ ]:
def train_adv(model, data_loader, test_loader,epsilon, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        accuracy = 0.0
        total = 0.0

        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            feats.requires_grad = True

            # for clean data
            optimizer.zero_grad()            
            outputs = model(feats)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
            total += len(labels)

            # for adversarial examples
            feats = fgsm_attack(feats, epsilon, feats.grad.data)
            optimizer.zero_grad()            
            outputs = model(feats)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
            total += len(labels)

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
                
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
            del outputs 

        
        print("Accuracy in train", (accuracy/total)*100.0)
        if task == 'Classification':
            train_loss, train_acc = test_classify(model, data_loader)
            val_loss, val_acc = test_classify(model, test_loader)
            val_acc_attack = test_attack(model,test_loader,epsilon)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}\tVal Accuracy (Attack): {:.4f}'.
                  format(train_loss, train_acc*100.0, val_loss, val_acc*100.0, val_acc_attack*100))
            
        scheduler.step()

''' Test the attack against the model '''
def test_attack(model, test_loader, epsilon):
    model.eval()
    correct = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        feats.requires_grad = True

        outputs = model(feats)
        init_pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        if init_pred != labels.item():
            continue

        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()

        # Call FGSM Attack
        perturbed_data = fgsm_attack(feats, epsilon, feats.grad.data)
        outputs = model(perturbed_data) 
        
        # Check for success
        final_pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == labels.item():
            correct += 1
           
        del feats
        del labels
        del outputs
    
    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    model.train()
    return final_acc

In [ ]:
numEpochs = 20
num_feats = 3

learningRate = 0.15
weightDecay = 1e-3

hidden_sizes = [64, 128, 256, 512]
num_classes = len(trainset1.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = torch.load(base_classifier)
network.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.85)   

network.train()
train_adv(network, trainloader, testloader, 0.05)

ADVERSARIAL TRAINING (PGD)

In [ ]:
def train_adv(model, data_loader, test_loader,epsilon,alpha,steps,random_start, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        accuracy = 0.0
        total = 0.0

        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            feats.requires_grad = True

            # for clean data
            optimizer.zero_grad()            
            outputs = model(feats)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
            total += len(labels)

            # for adversarial examples
            feats = pgd_attack(feats, epsilon, feats.grad.data,alpha,steps,random_start)
            optimizer.zero_grad()            
            outputs = model(feats)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
            total += len(labels)

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
                
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
            del outputs #Can delete

        
        print("Accuracy in train", (accuracy/total)*100.0)
        if task == 'Classification':
            print("In")
            train_loss, train_acc = test_classify(model, data_loader)
            val_loss, val_acc = test_classify(model, test_loader)
            val_acc_attack = test_attack(model,test_loader,epsilon,alpha,steps,random_start)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}\tVal Accuracy (Attack): {:.4f}'.
                  format(train_loss, train_acc*100.0, val_loss, val_acc*100.0, val_acc_attack*100))
            
        scheduler.step()

def test_attack(model, test_loader, epsilon,alpha,steps,random_start):
    model.eval()
    correct = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        feats.requires_grad = True

        outputs = model(feats)
        init_pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        if init_pred != labels.item():
            continue

        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()


        # Call PGD attack 
        perturbed_data = pgd_attack(feats, epsilon, feats.grad.data,alpha,steps,random_start)
        outputs = model(perturbed_data) 
      
        
        # Check for success
        final_pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == labels.item():
            correct += 1
           
        del feats
        del labels
        del outputs
    
    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {} Alpha:{} \tTest Accuracy = {} / {} = {}".format(epsilon,alpha, correct, len(test_loader), final_acc))

    model.train()
    return final_acc


In [ ]:
defense_model = torch.load(fgsm_trained_model)
numEpochs = 50
num_feats = 3

learningRate = 0.15
weightDecay = 1e-3

hidden_sizes = [64, 128, 256, 512]
num_classes = len(trainset1.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


defense_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(defense_model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.85)   


epsilon = 0.3
random_start = True
alpha = 2/255
steps = 40
train_adv(defense_model,trainloader, testloader, epsilon,alpha,steps,random_start)

FGSM ATTACK ON DIFFERENT MODELS

In [ ]:
def test_attack(model, attack_set):
    model.eval()

    device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.CrossEntropyLoss()    

    test_loss = []
    accuracy = 0
    total = 0

    for i in range(len(attack_set[0])):

        feats = attack_set[0][i]
        labels = attack_set[1][i]
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        accuracy += torch.sum((torch.argmax(outputs,dim=1)==labels)).item()
        total += len(labels)

        del feats
        del labels
        del outputs

    print("Accuracy", accuracy/total)
    return accuracy/total

In [ ]:
attack_model = torch.load(attack_model)

In [ ]:
attack_images_fgsm = attack_images(attack_model,testloader, 0.15)

In [ ]:
defense_model = torch.load(base_classifier)
test_attack(defense_model,attack_images_fgsm)

Accuracy 0.3744


0.3744

In [ ]:
defense_model = torch.load(fgsm_trained_model)
test_attack(defense_model,attack_images_fgsm)

Accuracy 0.7206


0.7206

In [ ]:
defense_model = torch.load(pgd_trained_model)
test_attack(defense_model,attack_images_fgsm)

Accuracy 0.7168


0.7168

In [ ]:
defense_model = torch.load(fgsm_pgd_trained_model)
test_attack(defense_model,attack_images_fgsm)

Accuracy 0.7116


0.7116

PGD ATTACK ON DIFFERENT MODELS

In [ ]:
attack_images_pgd = attack_images_pgd(attack_model,testloader)

In [ ]:
defense_model = torch.load(base_classifier)
test_attack(defense_model,attack_images_pgd)

Accuracy 0.3129


0.3129

In [ ]:
defense_model = torch.load(fgsm_trained_model)
test_attack(defense_model,attack_images_pgd)

Accuracy 0.3758


0.3758

In [ ]:
defense_model = torch.load(pgd_trained_model)
test_attack(defense_model,attack_images_pgd)

Accuracy 0.6181


0.6181

In [ ]:
defense_model = torch.load(fgsm_pgd_trained_model)
test_attack(defense_model,attack_images_pgd)

Accuracy 0.6457


0.6457

In [87]:
!cd ./gdrive/My\ Drive/

In [100]:
!ls

data  gdrive  sample_data


In [105]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.config/
	data/
	gdrive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


In [104]:
!ls

data  gdrive  sample_data
